In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [50]:
train_df = pd.read_csv("mitbih_train.csv", header=None)
test_df  = pd.read_csv("mitbih_test.csv", header=None)

X_train = train_df.iloc[:, :-1].values
y_train = train_df.iloc[:, -1].values

X_test  = test_df.iloc[:, :-1].values
y_test  = test_df.iloc[:, -1].values


In [51]:
X_train 

array([[0.97794116, 0.92647058, 0.68137252, ..., 0.        , 0.        ,
        0.        ],
       [0.96011394, 0.86324787, 0.46153846, ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.65945947, 0.18648648, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.90612245, 0.62448978, 0.59591836, ..., 0.        , 0.        ,
        0.        ],
       [0.85822785, 0.64556962, 0.84556961, ..., 0.        , 0.        ,
        0.        ],
       [0.90150636, 0.84588647, 0.80069524, ..., 0.        , 0.        ,
        0.        ]])

In [52]:
train_df

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,0.977941,0.926471,0.681373,0.245098,0.154412,0.191176,0.151961,0.085784,0.058824,0.049020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.960114,0.863248,0.461538,0.196581,0.094017,0.125356,0.099715,0.088319,0.074074,0.082621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000000,0.659459,0.186486,0.070270,0.070270,0.059459,0.056757,0.043243,0.054054,0.045946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.925414,0.665746,0.541436,0.276243,0.196133,0.077348,0.071823,0.060773,0.066298,0.058011,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.967136,1.000000,0.830986,0.586854,0.356808,0.248826,0.145540,0.089202,0.117371,0.150235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87549,0.807018,0.494737,0.536842,0.529825,0.491228,0.484211,0.456140,0.396491,0.284211,0.136842,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
87550,0.718333,0.605000,0.486667,0.361667,0.231667,0.120000,0.051667,0.001667,0.000000,0.013333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
87551,0.906122,0.624490,0.595918,0.575510,0.530612,0.481633,0.444898,0.387755,0.322449,0.191837,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
87552,0.858228,0.645570,0.845570,0.248101,0.167089,0.131646,0.121519,0.121519,0.118987,0.103797,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [53]:
y_test

array([0., 0., 0., ..., 4., 4., 4.])

## Apply Fourier Transform

In [54]:
def fft_denoise(ecg, fs=360):
    # Convert time → frequency
    fft_vals = np.fft.fft(ecg)
    freqs = np.fft.fftfreq(len(fft_vals), d=1/fs)

    # Keep only ECG frequencies
    fft_vals[np.abs(freqs) < 0.5] = 0     # baseline wander
    fft_vals[np.abs(freqs) > 40] = 0      # muscle + power-line noise

    # Convert frequency → time
    clean_ecg = np.fft.ifft(fft_vals)
    return np.real(clean_ecg)


## Modify the Dataset Using FFT

In [55]:
X_train_fft = np.array([fft_denoise(x) for x in X_train])
X_test_fft  = np.array([fft_denoise(x) for x in X_test])

### Normalize and Reshape

In [56]:
X_train_fft = X_train_fft / np.max(X_train_fft)
X_test_fft  = X_test_fft / np.max(X_test_fft)

X_train_fft = X_train_fft.reshape(X_train_fft.shape[0], 187, 1)
X_test_fft  = X_test_fft.reshape(X_test_fft.shape[0], 187, 1)


In [57]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=5)
y_test  = to_categorical(y_test, num_classes=5)


In [58]:
print("X_train:", X_train_fft.shape)
print("y_train:", y_train.shape)
print("X_test :", X_test_fft.shape)
print("y_test :", y_test.shape)


X_train: (87554, 187, 1)
y_train: (87554, 5)
X_test : (21892, 187, 1)
y_test : (21892, 5)


### Deep Learning Model (CNN-LSTM)

In [59]:
model = Sequential([
    Conv1D(32, 5, activation='relu', input_shape=(187,1)),
    MaxPooling1D(2),

    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(2),

    LSTM(64),
    Dropout(0.3),

    Dense(32, activation='relu'),
    Dense(5, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


### Train Model

In [60]:
history = model.fit(
    X_train_fft, y_train,
    validation_split=0.2,
    epochs=15,
    batch_size=64
)


Epoch 1/15
1094/1095 [============================>.] - ETA: 0s - loss: 0.0227 - accuracy: 0.9988WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x000001CB02CCCEE0> and will run it as-is.
Cause: Unable to locate the source code of <function Model.make_test_function.<locals>.test_function at 0x000001CB02CCCEE0>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function Model.make_test_function.<locals>.test_function at 0x000001CB02CCCEE0>. Note that functions defined in certain environments, like the int

### Eval

In [63]:
train_acc = history.history['accuracy'][-1]
val_acc   = history.history['val_accuracy'][-1]

print("Training Accuracy :", train_acc)
print("Validation Accuracy:", val_acc)


Training Accuracy : 1.0
Validation Accuracy: 0.13865570724010468


In [64]:
test_loss, test_accuracy = model.evaluate(X_test_fft, y_test)
print("Test Accuracy:", test_accuracy)


685/685 [==============================] - 14s 20ms/step - loss: 4.3669 - accuracy: 0.8276
Test Accuracy: 0.8276082873344421
